<a href="https://colab.research.google.com/github/steveny1989/DeepLearningExamples/blob/master/poem_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer

#加载编码器
tokenizer = AutoTokenizer.from_pretrained('uer/gpt2-chinese-cluecorpussmall')

print(tokenizer)

PreTrainedTokenizerFast(name_or_path='uer/gpt2-chinese-cluecorpussmall', vocab_size=21128, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [ ]:
tokenizer.batch_encode_plus([
    '床前明月光，疑是地上霜。'
])

{'input_ids': [[101, 2414, 1184, 3209, 3299, 1045, 8024, 4542, 3221, 1765, 677, 7458, 511, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
#简单数据集
class Dataset(torch.utils.data.Dataset):

    def __init__(self):
        with open('libai.txt') as f:
            lines = f.readlines()
        lines = [i.strip() for i in lines]

        self.lines = lines

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, i):
        return self.lines[i]

        
dataset = Dataset()

len(dataset), dataset[4]

(5501, '自从建安来，绮丽不足珍。圣代复元古，垂衣贵清真。')

In [ ]:
import torch
import os
import pandas as pd

In [ ]:
#更多数据数据集
class Dataset(torch.utils.data.Dataset):

    def __init__(self):
        data = []

        data.append(pd.read_csv('唐.csv'))

        data = pd.concat(data).reset_index()

        data = data['内容']

        data = data.str.strip()

        #移除一些标点符号
        data = data.str.replace('[《》“”「」]', '', regex=True)

        #正则过滤
        select = data.str.match('^[\w，。？、！：；]+$', na=False)
        data = data[select]

        #标点符号合并
        data = data.str.replace('[？！；]', '。', regex=True)
        data = data.str.replace('[、：]', '，', regex=True)

        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data.iloc[i]


dataset = Dataset()

len(dataset), dataset[0]

UnicodeDecodeError: ignored

In [ ]:
def collate_fn(data):
    data = tokenizer.batch_encode_plus(data,
                                       padding=True,
                                       truncation=True,
                                       max_length=512,
                                       return_tensors='pt')

    data['labels'] = data['input_ids'].clone()

    return data

#数据加载器
loader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=8,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
)

for i, data in enumerate(loader):
    break

for k, v in data.items():
    print(k, v.shape)

len(loader)

input_ids torch.Size([8, 26])
token_type_ids torch.Size([8, 26])
attention_mask torch.Size([8, 26])
labels torch.Size([8, 26])


687

In [ ]:
from transformers import AutoModelForCausalLM, GPT2Model

#加载模型
model = AutoModelForCausalLM.from_pretrained(
    'uer/gpt2-chinese-cluecorpussmall')

#统计参数量
print(sum(i.numel() for i in model.parameters()) / 10000)

with torch.no_grad():
    out = model(**data)

out['loss'], out['logits'].shape

Downloading:   0%|          | 0.00/421M [00:00<?, ?B/s]

10206.8736


(tensor(7.0028), torch.Size([8, 26, 21128]))

In [ ]:
def generate(text, row, col):

    def generate_loop(data):
        with torch.no_grad():
            out = model(**data)

        #取最后一个字
        #[5, b, 50257]
        out = out['logits']
        #[5, 50257]
        out = out[:, -1]

        #第50大的值,以此为分界线,小于该值的全部赋值为负无穷
        #[5, 50257] -> [5, 50]
        topk_value = torch.topk(out, 50).values
        #[5, 50] -> [5] -> [5, 1]
        topk_value = topk_value[:, -1].unsqueeze(dim=1)

        #赋值
        #[5, 50257]
        out = out.masked_fill(out < topk_value, -float('inf'))

        #不允许写特殊符号
        out[:, tokenizer.sep_token_id] = -float('inf')
        out[:, tokenizer.unk_token_id] = -float('inf')
        out[:, tokenizer.pad_token_id] = -float('inf')
        for i in '，。':
            out[:, tokenizer.get_vocab()[i]] = -float('inf')

        #根据概率采样,无放回,所以不可能重复
        #[5, 50257] -> [5, 1]
        out = out.softmax(dim=1)
        out = out.multinomial(num_samples=1)

        #强制添加标点符号
        c = data['input_ids'].shape[1] / (col + 1)
        if c % 1 == 0:
            if c % 2 == 0:
                out[:, 0] = tokenizer.get_vocab()['。']
            else:
                out[:, 0] = tokenizer.get_vocab()['，']

        data['input_ids'] = torch.cat([data['input_ids'], out], dim=1)
        data['attention_mask'] = torch.ones_like(data['input_ids'])
        data['token_type_ids'] = torch.zeros_like(data['input_ids'])
        data['labels'] = data['input_ids'].clone()

        if data['input_ids'].shape[1] >= row * col + row + 1:
            return data

        return generate_loop(data)

    #重复5遍
    data = tokenizer.batch_encode_plus([text] * 10, return_tensors='pt')
    data['input_ids'] = data['input_ids'][:, :-1]
    data['attention_mask'] = torch.ones_like(data['input_ids'])
    data['token_type_ids'] = torch.zeros_like(data['input_ids'])
    data['labels'] = data['input_ids'].clone()

    data = generate_loop(data)

    for i in range(10):
        print(i, tokenizer.decode(data['input_ids'][i]))

    print('Generated by AI')

generate('兔年吉祥', row=4, col=5)


0 [CLS] 兔 年 吉 祥 庆 ， 江 苏 盐 城 人 。 清 朝 末 十 五 ， 著 名 通 判 刑 。
1 [CLS] 兔 年 吉 祥 物 ， 出 自 《 古 罗 。 文 字 志 》 里 ， 作 者 是 兔 子 。
2 [CLS] 兔 年 吉 祥 / ， 可 以 指 ： < 。 < onlyinclude ， " < onlyincl 。
3 [CLS] 兔 年 吉 祥 ： ， 又 称 兔 年 吉 。 是 日 食 性 动 ， 动 物 可 以 食 。
4 [CLS] 兔 年 吉 祥 为 ， 是 日 本 动 画 。 原 创 作 品 的 ， 于 1997 年 于 台 。
5 [CLS] 兔 年 吉 祥 物 ， 又 译 作 狸 年 。 猫 年 的 吉 祥 ， 是 在 公 元 2017 。
6 [CLS] 兔 年 吉 祥 物 ， 又 称 兔 年 兔 。 兔 年 ： 日 本 ， 兔 年 ： 太 岁 。
7 [CLS] 兔 年 吉 祥 物 ， 或 者 兔 年 吉 。 兔 年 又 称 兔 ， 本 年 兔 年 兔 。
8 [CLS] 兔 年 吉 祥 物 ， 是 日 本 的 动 。 最 后 一 个 字 ， 是 兔 肉 的 意 。
9 [CLS] 兔 年 吉 祥 物 ， 中 华 民 国 特 。 兔 年 吉 祥 物 ， 中 华 民 国 特 。
Generated by AI


In [ ]:
from transformers import AdamW
from transformers.optimization import get_scheduler


#训练
def train():
    global model
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)

    optimizer = AdamW(model.parameters(), lr=5e-5)
    scheduler = get_scheduler(name='linear',
                              num_warmup_steps=0,
                              num_training_steps=len(loader),
                              optimizer=optimizer)

    model.train()
    for i, data in enumerate(loader):
        for k in data.keys():
            data[k] = data[k].to(device)
        out = model(**data)
        loss = out['loss']

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        optimizer.zero_grad()
        model.zero_grad()

        if i % 1000 == 0:
            labels = data['labels'][:, 1:]
            out = out['logits'].argmax(dim=2)[:, :-1]

            select = labels != 0
            labels = labels[select]
            out = out[select]
            del select

            accuracy = (labels == out).sum().item() / labels.numel()

            lr = optimizer.state_dict()['param_groups'][0]['lr']

            print(i, loss.item(), lr, accuracy)

    model = model.to('cpu')
    torch.save(model, 'save.model')


train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 9.550468444824219 4.9927219796215426e-05 0.12931034482758622


In [ ]:
model = torch.load('save.model')

generate('春', row = 1, col = 5)
generate('节', row = 1, col = 5)
generate('快', row = 1, col = 5)
generate('乐', row = 1, col = 5)



0 [CLS] 春 风 来 时 歌 ，
1 [CLS] 春 风 不 见 远 ，
2 [CLS] 春 风 落 白 云 ，
3 [CLS] 春 夜 夜 去 归 ，
4 [CLS] 春 色 入 清 风 ，
5 [CLS] 春 光 白 云 露 ，
6 [CLS] 春 风 拂 寒 雨 ，
7 [CLS] 春 风 来 何 时 ，
8 [CLS] 春 风 何 处 无 ，
9 [CLS] 春 风 落 日 闲 ，
Generated by AI
0 [CLS] 节 钺 高 马 蹄 ，
1 [CLS] 节 气 不 同 天 ，
2 [CLS] 节 度 万 乘 光 ，
3 [CLS] 节 日 歌 乐 相 ，
4 [CLS] 节 逢 秋 水 相 ，
5 [CLS] 节 气 不 动 光 ，
6 [CLS] 节 节 不 如 天 ，
7 [CLS] 节 奏 在 遥 夜 ，
8 [CLS] 节 操 惊 云 气 ，
9 [CLS] 节 气 在 西 游 ，
Generated by AI
0 [CLS] 快 哉 不 可 及 ，
1 [CLS] 快 乐 自 顾 开 ，
2 [CLS] 快 手 生 紫 微 ，
3 [CLS] 快 意 为 花 色 ，
4 [CLS] 快 速 行 乐 不 ，
5 [CLS] 快 乐 不 成 飞 ，
6 [CLS] 快 意 不 可 寻 ，
7 [CLS] 快 发 银 鞍 马 ，
8 [CLS] 快 哉 尔 自 嗟 ，
9 [CLS] 快 来 我 相 邀 ，
Generated by AI
0 [CLS] 乐 如 玉 树 花 ，
1 [CLS] 乐 天 不 应 见 ，
2 [CLS] 乐 陵 万 里 人 ，
3 [CLS] 乐 和 与 世 间 ，
4 [CLS] 乐 不 可 知 日 ，
5 [CLS] 乐 为 万 方 物 ，
6 [CLS] 乐 为 一 杯 酒 ，
7 [CLS] 乐 不 去 东 东 ，
8 [CLS] 乐 于 见 君 游 ，
9 [CLS] 乐 在 长 吟 君 ，
Generated by AI
